In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

In [0]:
data = pd.read_csv('train_preprocess.csv', sep=';')
city_names = data.Location.unique().tolist()

In [0]:
categorical_cols = [cat for cat in data.columns if data[cat].dtype == 'O']
numerical_cols = list(set(data.columns.values.tolist()) - set(categorical_cols))
numeric_means = data[numerical_cols].mean()
X_real_mean = data[numerical_cols].fillna(numeric_means)
X_cat = data[categorical_cols].fillna('NaN')
from sklearn.feature_extraction import DictVectorizer as DV
encoder = DV(sparse = False)
X_cat_oh = encoder.fit_transform(X_cat.astype('str').T.to_dict().values())

In [0]:
target = {city:data.RainTomorrow.loc[data.Location == city].reset_index(drop=True) for city in city_names}
data.drop('RainTomorrow', axis=1, inplace=True)
locations = {city:data.loc[data.Location == city].reset_index(drop=True) for city in city_names}

In [0]:
percent = {city:int(len(locations[city]) * 0.9) for city in city_names}

In [0]:
test_target = {city:target[city].iloc[percent[city]:].values for city in city_names}
target = {city:target[city].iloc[:percent[city]].values for city in city_names}
test_locations = {city:locations[city].iloc[percent[city]:, :] for city in city_names}
locations = {city:locations[city].iloc[:percent[city], :] for city in city_names}

In [107]:
locations['Albury'].head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,Year,Month,Day,WindSpeedDelta,WindDirChange,HumidityDelta,PressureDelta,CloudDelta,TempDelta,WindDir3pm=E,WindDir3pm=ENE,WindDir3pm=ESE,WindDir3pm=N,WindDir3pm=NE,WindDir3pm=NNE,WindDir3pm=NNW,WindDir3pm=NW,WindDir3pm=NaN,WindDir3pm=S,WindDir3pm=SE,WindDir3pm=SSE,WindDir3pm=SSW,WindDir3pm=SW,WindDir3pm=W,WindDir3pm=WNW,WindDir3pm=WSW,WindDir9am=E,WindDir9am=ENE,WindDir9am=ESE,WindDir9am=N,WindDir9am=NE,WindDir9am=NNE,WindDir9am=NNW,WindDir9am=NW,WindDir9am=NaN,WindDir9am=S,WindDir9am=SE,WindDir9am=SSE,WindDir9am=SSW,WindDir9am=SW,WindDir9am=W,WindDir9am=WNW,WindDir9am=WSW,WindGustDir=E,WindGustDir=ENE,WindGustDir=ESE,WindGustDir=N,WindGustDir=NE,WindGustDir=NNE,WindGustDir=NNW,WindGustDir=NW,WindGustDir=NaN,WindGustDir=S,WindGustDir=SE,WindGustDir=SSE,WindGustDir=SSW,WindGustDir=SW,WindGustDir=W,WindGustDir=WNW,WindGustDir=WSW
0,Albury,13.4,22.9,0.6,4.925332,7.229785,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.000000,4.640386,16.9,21.8,2008,12,1,4.0,True,-49.0,-0.6,0.110117,4.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Albury,7.4,25.1,0.0,4.925332,7.229785,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,4.531478,4.640386,17.2,24.3,2008,12,2,18.0,True,-19.0,-2.8,0.110117,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,Albury,12.9,25.7,0.0,4.925332,7.229785,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,4.531478,2.000000,21.0,23.2,2008,12,3,7.0,True,-8.0,1.1,0.110117,2.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,Albury,9.2,28.0,0.0,4.925332,7.229785,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,4.531478,4.640386,18.1,26.5,2008,12,4,-2.0,True,-29.0,-4.8,0.110117,8.4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Albury,17.5,32.3,1.0,4.925332,7.229785,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.000000,8.000000,17.8,29.7,2008,12,5,13.0,True,-49.0,-4.8,1.000000,11.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [0]:
for city in city_names:
    locations[city].drop('Location', axis=1, inplace=True)
    test_locations[city].drop('Location', axis=1, inplace=True)
    dataset = locations[city].values
    data_mean = dataset.mean()
    data_std = dataset.std()
    locations[city] = (dataset-data_mean) / data_std
    test_locations[city] = (test_locations[city].values - data_mean) / data_std

In [0]:
def multivariable_data(dataset, target, target_size):
    data=[]
    labels=[]
    for i in range(dataset.shape[0] - target_size):
    data.append(dataset[i:i+target_size, :])
    labels.append(target[i + target_size])

    return np.array(data), np.array(labels)

In [116]:
locations['Albury'].shape

(1900, 76)

In [119]:
multivariable_data(locations['Albury'], target['Albury'], 20)
print(s1.shape, s2.shape)

(1880, 20, 76) (1880,)


In [257]:
dataset = [multivariable_data(locations[city], target[city], 5) for city in city_names]
X_train = np.vstack((dataset[i][0] for i in range(len(dataset))))
y_train = np.hstack((dataset[i][1] for i in range(len(dataset))))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


In [258]:
dataset_test = [multivariable_data(test_locations[city], test_target[city], 5) for city in city_names]
X_test = np.vstack((dataset_test[i][0] for i in range(len(dataset_test))))
y_test = np.hstack((dataset_test[i][1] for i in range(len(dataset_test))))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
X_test = np.asarray(X_test).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)
X_train = np.asarray(X_train).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)

In [250]:
X_train.shape

(62903, 10, 25)

In [0]:
import keras

model = tf.keras.models.Sequential()
model.add( tf.keras.layers.SimpleRNN(64, dropout=0.2, return_sequences=True))
model.add( tf.keras.layers.SimpleRNN(64, dropout=0.2))
# model.add(tf.keras.layers.LSTM(64, dropout=0.2, input_shape=(X_train[0].shape), return_sequences=True))
# model.add(tf.keras.layers.LSTM(64, dropout=0.2))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [185]:
model.predict(np.expand_dims(X_test[0, :], 0))

array([[0.3714308]], dtype=float32)

In [297]:
model.build(input_shape=X_train.shape)
model.summary()

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_6 (SimpleRNN)     multiple                  5760      
_________________________________________________________________
simple_rnn_7 (SimpleRNN)     multiple                  8256      
_________________________________________________________________
dense_27 (Dense)             multiple                  1040      
_________________________________________________________________
dense_28 (Dense)             multiple                  17        
Total params: 15,073
Trainable params: 15,073
Non-trainable params: 0
_________________________________________________________________


In [300]:
model.fit(X_train, y_train, batch_size=256, epochs=30, validation_data=(X_test, y_test), shuffle=True)

Epoch 1/30
247/247 [==============================] - 5s 20ms/step - loss: 0.5425 - accuracy: 0.7681 - val_loss: 0.5574 - val_accuracy: 0.7574
Epoch 2/30
247/247 [==============================] - 5s 18ms/step - loss: 0.5405 - accuracy: 0.7694 - val_loss: 0.5654 - val_accuracy: 0.7574
Epoch 3/30
247/247 [==============================] - 5s 19ms/step - loss: 0.5405 - accuracy: 0.7694 - val_loss: 0.5569 - val_accuracy: 0.7574
Epoch 4/30
247/247 [==============================] - 5s 19ms/step - loss: 0.5403 - accuracy: 0.7694 - val_loss: 0.5542 - val_accuracy: 0.7574
Epoch 5/30
247/247 [==============================] - 5s 19ms/step - loss: 0.5398 - accuracy: 0.7694 - val_loss: 0.5530 - val_accuracy: 0.7574
Epoch 6/30
247/247 [==============================] - 5s 19ms/step - loss: 0.5389 - accuracy: 0.7694 - val_loss: 0.5501 - val_accuracy: 0.7574
Epoch 7/30
247/247 [==============================] - 5s 19ms/step - loss: 0.5381 - accuracy: 0.7694 - val_loss: 0.5511 - val_accuracy: 0.7574

KeyboardInterrupt: ignored

In [290]:
from sklearn.metrics import roc_auc_score, confusion_matrix
print(roc_auc_score(y_test, model.predict(X_test) >= 0.5))
print(confusion_matrix(y_test, model.predict(X_test) >= 0.5))

0.5080619568569572
[[5169   47]
 [1629   42]]
